In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [4]:
from accounts import Account

In [5]:
account = Account.get("Ed")
account

Account(name='ed', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [7]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promosing")

'Completed. Latest details:\n{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-09-03 09:16:19", "rationale": "Because this bookstore website looks promosing"}], "portfolio_value_time_series": [["2025-09-03 09:15:47", 10000.0], ["2025-09-03 09:16:19", 9954.604]], "total_portfolio_value": 9954.604, "total_profit_loss": -45.39600000000064}'

In [8]:
account.report()

'{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-09-03 09:16:19", "rationale": "Because this bookstore website looks promosing"}], "portfolio_value_time_series": [["2025-09-03 09:15:47", 10000.0], ["2025-09-03 09:16:19", 9954.604], ["2025-09-03 09:16:29", 10086.604]], "total_portfolio_value": 10086.604, "total_profit_loss": 86.60399999999936}'

In [9]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 66.132,
  'timestamp': '2025-09-03 09:16:19',
  'rationale': 'Because this bookstore website looks promosing'}]

### Now we write an MCP server and use it directly!

In [4]:
# Now let's use our accounts server as an MCP Server
# So Basically, this MCP Server means that we run the file python that have decoratoor tools and wrap in in server MCP as tool

params = {"command": "uv", "args": ["run", "accounts_server.py"]} # Like CMD uv run accounts_server.py
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

In [13]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [14]:
instructions = "You are able to manage account for a client, and answer questions about the account"
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [15]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    
    # In server we add the agent
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your current cash balance is $9,801.60. In your holdings, you have 3 shares of AMZN. If you need any further details or want to make any transactions, just let me know!

### Now let's build our own MCP Client

In [ ]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'get_balanceOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema={'additionalProperties': {'type': 'integer'}, 'title': 'get_holdingsDictOutput', 'type': 'object'}, annotations=None, meta=None), Tool(name='buy_shares', title=None, description="B

In [17]:
request = "My name is Ed and my account is under the name Ed. What's my balance?"

# Actually we must use like this for MCP, so the client will search for the server mcp
# So when we call the tools that already wrapper by the MCP, it will create the MCP Client with MCP Server inside it 

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools) 
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your account balance is $9,801.60. If you need any other information or want to make any transactions, just let me know!

In [18]:
context = await read_accounts_resource("ed")
print(context)

# So resource in MCP means that we wrap the function business logic to the MCP Server to be avalailable at a certain resource and we can expose it
# The purpose is to share with other people about your resources and do not need to understand the business logic works

{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-09-03 09:16:19", "rationale": "Because this bookstore website looks promosing"}], "portfolio_value_time_series": [["2025-09-03 09:15:47", 10000.0], ["2025-09-03 09:16:19", 9954.604], ["2025-09-03 09:16:29", 10086.604], ["2025-09-03 09:50:51", 9822.604]], "total_portfolio_value": 9822.604, "total_profit_loss": -177.39600000000064}


In [19]:
from accounts import Account

Account.get("ed").report()

'{"name": "ed", "balance": 9801.604, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 66.132, "timestamp": "2025-09-03 09:16:19", "rationale": "Because this bookstore website looks promosing"}], "portfolio_value_time_series": [["2025-09-03 09:15:47", 10000.0], ["2025-09-03 09:16:19", 9954.604], ["2025-09-03 09:16:29", 10086.604], ["2025-09-03 09:50:51", 9822.604], ["2025-09-03 09:51:32", 9867.604]], "total_portfolio_value": 9867.604, "total_profit_loss": -132.39600000000064}'

Bad pipe message: %s [b'\xdal\x05\x8d\xf2\x9b6\x0e\x90\x9fZ\x1fue\xeal|\xdf\x00\x01|\x00\x00\x00\x01\x00\x02\x00\x03\x00\x04\x00\x05\x00\x06\x00\x07\x00\x08\x00\t\x00\n\x00\x0b\x00\x0c\x00\r\x00\x0e\x00\x0f\x00\x10\x00\x11\x00\x12\x00\x13\x00\x14\x00\x15\x00\x16\x00\x17\x00\x18\x00\x19\x00\x1a\x00\x1b\x00/\x000\x001\x002\x003\x004\x005\x006\x007\x008\x009\x00:\x00;\x00<\x00=\x00>\x00?\x00@\x00A\x00B\x00C\x00D\x00E\x00F\x00g\x00h\x00i\x00j\x00k\x00l\x00m\x00\x84\x00', b'\x86\x00\x87\x00\x88\x00\x89\x00\x96\x00\x97\x00\x98\x00\x99\x00\x9a\x00\x9b\x00\x9c\x00\x9d\x00\x9e\x00\x9f\x00\xa0\x00\xa1\x00\xa2\x00\xa3\x00\xa4\x00\xa5\x00\xa6\x00\xa7\x00\xba\x00\xbb\x00\xbc\x00\xbd\x00\xbe\x00\xbf\x00\xc0\x00\xc1\x00\xc2\x00\xc3\x00\xc4\x00\xc5\x13\x01\x13\x02\x13\x03\x13\x04\x13\x05\xc0\x01\xc0\x02\xc0\x03\xc0\x04\xc0\x05\xc0\x06\xc0\x07\xc0\x08\xc0\t\xc0\n\xc0\x0b\xc0\x0c\xc0\r\xc0\x0e\xc0\x0f\xc0\x10\xc0\x11\xc0\x12\xc0\x13\xc0\x14\xc0\x15\xc0\x16\xc0\x17\xc0\x18\xc0\x19\xc0#\xc0$\xc0']
Bad pip